In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from datetime import datetime
import pandas as pd

In [15]:
# ubicacion de chrome driver y la url que queremos utilizar
PATH_TO_DRIVER = 'C:/chromedriver-win32/chromedriver-win32/chromedriver.exe'
url = 'https://www.guatecompras.gt/reportes/Rpt_ProveedoresPorMonto.aspx'

In [5]:
if False:
    try: 
        # esperamos a que la pagina cargue bien 
        wait = WebDriverWait(driver, 10)

        # fecha de interes (mes y anio)
        anio_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlAnioA')
        anio = Select(anio_dropdown)
        anio.select_by_value('2005')    # 2005 - actual
        print('cambio el año...')
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'MasterGC_ContentBlockHolder_ddlMesA'))  # Esperar que el siguiente elemento esté presente
        )
        
        mes_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlMesA')
        mes = Select(mes_dropdown)
        mes.select_by_value('05')   # 01-12 meses y -1 para todos
        print('Cambio mes...')
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'MasterGC_ContentBlockHolder_ddlRango'))  # Esperar que el siguiente elemento esté presente
        )


        # rango de montos (nosotros tomamos para la primera prueba solo CPEs)
        rango_monto_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlRango')
        rango_monto = Select(rango_monto_dropdown)
        rango_monto.select_by_value('2')    # 0: [0.01, 90000.00], 1: [90000.01, 900000.00], 2: [900000.01, inf]
        print('Cambio rango...')
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'MasterGC_ContentBlockHolder_ddlTipoProveedor'))  # Esperar que el siguiente elemento esté presente
        )


        # tipo de proveedor (queremos ambos, con y sin NIT)
        tipo_proveedor_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlTipoProveedor')
        tipo_proveedor = Select(tipo_proveedor_dropdown)
        tipo_proveedor.select_by_value('0')     # 0: Con NIT, 1: Sin NIT
        print('Cambio tipo proveedor...')
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'MasterGC_ContentBlockHolder_ddlTipoOrganizacion'))  # Esperar que el siguiente elemento esté presente
        )


        # Tipos de organización (queremos todas)
        tipo_org_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlTipoOrganizacion')
        tipo_org = Select(tipo_org_dropdown)
        tipo_org.select_by_value('-1')      # -1: Todos los tipos de organización
        print('Cambio tipo organizacion...')
        time.sleep(3)
        type = {0: 'MasterGC_ContentBlockHolder_BtnExcel', 1: 'MasterGC_ContentBlockHolder_BtnCSV'}
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, type.get(1)))  # Esperar que el siguiente elemento esté presente
        )


        # para asegurarnos que todo este bien y la pagina termine de actualizarse antes de realizar el click
        #time.sleep(3)
        #wait = WebDriverWait(driver, 3)


        # boton de descarga, los almacenaremos en excel o csv
        boton_descarga = driver.find_element(By.ID, type.get(1))
        boton_descarga.click()
        print('Lo descargamos...')


        # Esperamos a que se descargue
        time.sleep(15)
    except Exception as e:
        print(f'Error: {e}')
    finally:
        driver.quit()
        print('salimos del driver')


In [6]:
hoy = datetime.today()
year = hoy.year

In [16]:
def guateMontosCPE(year : int, month : str = '-1', interval : str = '2', NIT : str = '0', output_file_type : int = 1) -> None:
    # creamos el objeto service
    service = Service(PATH_TO_DRIVER)

    # se inicializa el navegador con el servicio
    driver = webdriver.Chrome(service=service)

    # esperamos a que la ventana cargue bien 
    WebDriverWait(driver, 5)

    # vamos a la página
    driver.get(url)

    
    try: 
        # esperamos a que la pagina cargue bien 
        WebDriverWait(driver, 10)

        # fecha de interes (mes y anio)
        anio_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlAnioA')
        anio = Select(anio_dropdown)
        anio.select_by_value(str(year))    # 2005 - actual
        print('cambio el año...')
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'MasterGC_ContentBlockHolder_ddlMesA'))  # Esperar que el siguiente elemento esté presente
        )
        
        mes_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlMesA')
        mes = Select(mes_dropdown)
        mes.select_by_value(month)   # 01-12 meses y -1 para todos
        print('Cambio mes...')
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'MasterGC_ContentBlockHolder_ddlRango'))  # Esperar que el siguiente elemento esté presente
        )


        # rango de montos (nosotros tomamos para la primera prueba solo CPEs)
        rango_monto_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlRango')
        rango_monto = Select(rango_monto_dropdown)
        rango_monto.select_by_value(interval)    # 0: [0.01, 90000.00], 1: [90000.01, 900000.00], 2: [900000.01, inf]
        print('Cambio rango...')
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'MasterGC_ContentBlockHolder_ddlTipoProveedor'))  # Esperar que el siguiente elemento esté presente
        )


        # tipo de proveedor (queremos ambos, con y sin NIT)
        tipo_proveedor_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlTipoProveedor')
        tipo_proveedor = Select(tipo_proveedor_dropdown)
        tipo_proveedor.select_by_value(NIT)     # 0: Con NIT, 1: Sin NIT
        print('Cambio tipo proveedor...')
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'MasterGC_ContentBlockHolder_ddlTipoOrganizacion'))  # Esperar que el siguiente elemento esté presente
        )


        # Tipos de organización (queremos todas)
        tipo_org_dropdown = driver.find_element(By.ID, 'MasterGC_ContentBlockHolder_ddlTipoOrganizacion')
        tipo_org = Select(tipo_org_dropdown)
        tipo_org.select_by_value('-1')      # -1: Todos los tipos de organización
        print('Cambio tipo organizacion...')
        time.sleep(3)
        type = {0: 'MasterGC_ContentBlockHolder_BtnExcel', 1: 'MasterGC_ContentBlockHolder_BtnCSV'}
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, type.get(output_file_type)))  # Esperar que el siguiente elemento esté presente
        )


        # para asegurarnos que todo este bien y la pagina termine de actualizarse antes de realizar el click
        #time.sleep(3)
        #wait = WebDriverWait(driver, 3)


        # boton de descarga, los almacenaremos en excel o csv
        boton_descarga = driver.find_element(By.ID, type.get(output_file_type))
        boton_descarga.click()
        print('Lo descargamos...')


        # Esperamos a que se descargue
        time.sleep(15)
    except Exception as e:
        print(f'Error: {e}')
    finally:
        driver.quit()
        print('salimos del driver')


In [17]:
guateMontosCPE(year=2012)

cambio el año...
Cambio mes...
Cambio rango...
Cambio tipo proveedor...
Cambio tipo organizacion...
Lo descargamos...
salimos del driver


In [12]:
for year in range(2022, 2025):
    print(year)
    guateMontosCPE(year=year)

2022
cambio el año...
Cambio mes...
Cambio rango...
Cambio tipo proveedor...
Cambio tipo organizacion...
Lo descargamos...
salimos del driver
2023
cambio el año...
Cambio mes...
Cambio rango...
Cambio tipo proveedor...
Cambio tipo organizacion...
Lo descargamos...
salimos del driver
2024
cambio el año...
Cambio mes...
Cambio rango...
Cambio tipo proveedor...
Cambio tipo organizacion...
Lo descargamos...
salimos del driver
